In [70]:
import numpy as np
rng = np.random.default_rng()

In [71]:
def heatbath_update(U,beta):
    site = tuple(rng.integers(0,len(U),4))
    theta = sample_angle(beta)
    U[site] = np.exp(1j*theta)

def sample_angle(beta):
    alpha = np.sqrt(2*beta)*beta

    while True:
        Z = rng.uniform(0,1)
        x = -1 + np.log(1 + np.exp(2*alpha - 1)*Z)

        Q = np.exp(alpha*(np.cos(np.pi/2*(1-x))-x))
        Q_max = np.exp(0.2105137*alpha)

        Z_prime = rng.uniform(0,1)
        if Q/Q_max > Z_prime:
            angle = np.pi*(1-x)/2
            return angle

def run_heatbath(U, beta, n, loop_sites):
    wilson_loop_sum = 0.0
    for _ in range(n):
        heatbath_update(U, beta)
        loop_value = wilson_loop(U, loop_sites)
        wilson_loop_sum += loop_value
    return wilson_loop_sum / n

def create_plaquette(x0=0, y0=0, z0=0, t0=0):
    loop_sites = [
        (x0, y0, z0, t0, 0),
        (x0+1, y0, z0, t0, 1),
        (x0, y0+1, z0, t0, 2),
        (x0, y0, z0, t0, 3),
    ]
    return loop_sites

def wilson_loop(U, loop_sites):
    loop_product = 1.0 + 0.0j
    for (x, y, z, t, mu) in loop_sites:
        loop_product *= U[(x, y, z, t)]
    return np.real(loop_product)

In [77]:
width = 4
num_sites = width*width*width*width
betas = np.linspace(0.0,2.0,21)
equil_sweeps = 1
measure_sweeps = 1
measurements = 2

for beta in betas:
    U = np.exp(2j * np.pi * np.random.rand(width,width,width,width))
    run_heatbath(U,beta,equil_sweeps*num_sites)

    for _ in range(measurements):
        run_heatbath(U,beta,measure_sweeps * num_sites)
        # measure